In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [475]:
test_data = pd.read_csv('test.csv')
test_data.head()

,Unnamed: 0,latitude,longitude,company,is_local,type,fin_1,fin_2,fin_3,fin_4,target
0,160000,39.284310,-76.735350,18187,1,1,113033.389907,0.000000,270.906219,31222.780176,0
1,160001,36.758509,-76.344861,11208,0,0,-87239.590275,73759.387510,759.194862,237587.544996,1
2,160002,43.402802,-75.217100,7437,1,4,-122084.498620,15528.109943,1039.655934,29612.346982,0
3,160003,41.871160,-87.848570,17362,0,1,87355.127256,0.000000,214.594205,163526.475818,1
4,160004,42.161296,-88.129184,11515,0,3,33014.437946,0.000000,379.819724,67499.397999,1


In [14]:
train_data = pd.read_csv('train.csv')
train_data.head()

,Unnamed: 0,latitude,longitude,company,is_local,type,fin_1,fin_2,fin_3,fin_4,target
0,0,40.10891,-83.09286,8336,0,3,-135060.089443,86013.396489,1206.094242,52287.082257,0
1,1,39.86542,-84.06280,18403,1,0,-1766.845055,14985.640180,477.494992,168836.215743,1
2,2,39.10266,-84.52468,14022,0,3,-177302.873693,44881.958005,1463.339889,130388.243325,0
3,3,39.10148,-84.52341,11051,0,0,209049.997460,0.000000,95.340075,103267.727546,1
4,4,41.06213,-81.53784,3243,0,3,8669.269507,0.000000,399.421926,177532.206618,1


In [480]:
from tensorflow.keras.layers import Embedding, Input, Flatten, Dense, BatchNormalization, Concatenate, Reshape
from tensorflow.keras.models import Model

In [386]:
#replacing companies that have <= 10 units with -1 
train_data.loc[train_data.company <= 10, ('company')] = -1
print(train_data['company'].unique().size)

3683


In [387]:
#clustering units by geographical location
features = train_data[['latitude', 'longitude']]
from sklearn.cluster import KMeans
kmeans_clustering = KMeans(
    init="random",
    n_clusters=300,
    n_init=10,
    max_iter=300,
    random_state=42
)
kmeans_clustering.fit(features)
labels = kmeans_clustering.labels_
train_data['cluster'] = labels

In [493]:
data_input = Input(shape=(5, ), name='features')
company_input = Input(shape=(1, ), name='company')
type_input = Input(shape=(1, ), name='type')
location_input = Input(shape=(1, ), name='location')

In [389]:
train_data['type'].size

160001

In [505]:
#replacing companies that have <= 10 units with -1 
#train_data.loc[train_data.company <= 10, ('company')] = -1
company_input = Input(shape=(1, ), name='company')
company_features = Embedding(3683, 64)(company_input)
company_features = Dense(32, activation="relu")(company_features)

In [496]:
#train_data['type'].unique().size
type_features = Embedding(train_data['type'].unique().size, 8)(type_input)

In [497]:
data_features = Dense(32, activation="relu")(data_input)
data_features = BatchNormalization()(data_features)
data_features = Dense(32, activation="relu")(data_input)
data_features = BatchNormalization()(data_features)

In [350]:
#clustering units by geographical location
features = train_data[['latitude', 'longitude']]
from sklearn.cluster import KMeans
kmeans_clustering = KMeans(
    init="random",
    n_clusters=300,
    n_init=10,
    max_iter=300,
    random_state=42
)
kmeans_clustering.fit(features)
labels = kmeans_clustering.labels_
train_data['cluster'] = labels
print(train_data['cluster'].size)

160001


In [498]:
location_features = Embedding(300, 32)(location_input)
location_features = Dense(32, activation="relu")(location_features)

In [210]:
#from tensorflow.keras import layers
#feat_conc = Concatenate()([data_features, company_features, type_features, location_features])

In [211]:
#feat_conc = Dense(64)(feat_conc)
#feat_conc = BatchNormalization()(feat_conc)
#feat_conc = Dense(64)(feat_conc)
#feat_conc = BatchNormalization()(feat_conc)
#feat_conc = Dense(1)(feat_conc)


In [212]:
from tensorflow import keras
#model = keras.Model(inputs=[data_input, company_input, type_input, location_input], outputs = [feat_conc])
#model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           [(None, 1, 4)]       0                                            
__________________________________________________________________________________________________
company (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
location (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_68 (Dense)                (None, 1, 32)        160         features[0][0]                   
____________________________________________________________________________________________

In [236]:
model1 = keras.Model(inputs=[data_input, company_input, type_input, location_input], outputs=[data_features])

In [237]:
model1.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense_79 (Dense)                (None, 32)           160         features[0][0]                   
__________________________________________________________________________________________________
company (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
type (InputLayer)               [(None, None)]       0                                            
___________________________________________________________________________________________

In [223]:
model2 = keras.Model(inputs=[data_input, company_input, type_input, location_input], outputs=[company_features])

In [224]:
model2.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
company (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 1, 64)        10240064    company[0][0]                    
__________________________________________________________________________________________________
features (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
type (InputLayer)               [(None, 1)]          0                                            
____________________________________________________________________________________________

In [225]:
model3 = keras.Model(inputs=[data_input, company_input, type_input, location_input], outputs=[type_features])

In [226]:
model3.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
type (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
features (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
company (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
location (InputLayer)           [(None, 1)]          0                                            
___________________________________________________________________________________________

In [228]:
model4 = keras.Model(inputs=[data_input, company_input, type_input, location_input], outputs=[location_features])

In [229]:
model4.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
location (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_22 (Embedding)        (None, 1, 32)        5120032     location[0][0]                   
__________________________________________________________________________________________________
features (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
company (InputLayer)            [(None, 1)]          0                                            
___________________________________________________________________________________________

In [269]:
print(type(company_features))
print(company_features)

<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("dense_81/Identity:0", shape=(None, 1, 32), dtype=float32)


In [506]:
reshaped_company_features = tf.reshape(company_features, [-1, 32])

In [507]:
reshaped_type_features = tf.reshape(type_features, [-1, 8])
reshaped_location_features = tf.reshape(location_features, [-1, 32])

In [508]:
from tensorflow.keras import layers
feat_conc = Concatenate()([data_features, reshaped_company_features, reshaped_type_features, reshaped_location_features])

In [509]:
feat_conc = Dense(64, activation="relu")(feat_conc)
feat_conc = BatchNormalization()(feat_conc)
feat_conc = Dense(64, activation="relu")(feat_conc)
feat_conc = BatchNormalization()(feat_conc)
feat_conc = Dense(1, name="targets", activation="relu")(feat_conc)

In [510]:
from tensorflow import keras
model = keras.Model(inputs=[data_input, company_input, type_input, location_input], outputs = [feat_conc])
model.summary()

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
company (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
location (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
features (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
embedding_45 (Embedding)        (None, 1, 64)        235712      company[0][0]                    
___________________________________________________________________________________________

In [399]:
test_data.loc[train_data.company <= 10, ('company')] = -1

In [476]:
features = test_data[['fin_1', 'fin_2', 'fin_3', 'fin_4', 'is_local']].values
company = test_data['company'].unique()
types = test_data['type'].unique()
#targets = test_data['target'].values[:300]

In [467]:
features_2 = test_data[['latitude', 'longitude']]
from sklearn.cluster import KMeans
kmeans_clustering = KMeans(
    init="random",
    n_clusters=300,
    n_init=10,
    max_iter=300,
    random_state=42
)
kmeans_clustering.fit(features_2)
labels = kmeans_clustering.labels_
test_data['cluster'] = labels
location = test_data['cluster'].values

In [403]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[
        keras.losses.BinaryCrossentropy(from_logits=True),
    ]
)

In [470]:
model.fit(
    {"features": features, "company": company, "type": types, "location": location},
    #{"targets": targets},
    epochs=10,
    batch_size=128,
)

ValueError: Data cardinality is ambiguous:
  x sizes: 40000, 2970, 5, 40000
Please provide data which shares the same first dimension.

In [428]:
print(company.size)

2939


In [477]:
company_mapping = {num:val for num, val in enumerate(company)}
type_mapping = {num:val for num, val in enumerate(types)}

In [478]:
print(company_mapping)
print(type_mapping)
#print(types.size)
#test_data['company'].map(company_mapping)

{0: 18187, 1: 11208, 2: 7437, 3: 17362, 4: 11515, 5: 8562, 6: 13600, 7: 16654, 8: 547, 9: 6401, 10: 4650, 11: 6925, 12: 16190, 13: 8755, 14: 7465, 15: 10589, 16: 5237, 17: 12287, 18: 700, 19: 7560, 20: 9825, 21: 10208, 22: 1265, 23: 16144, 24: 8516, 25: 9384, 26: 18485, 27: 13595, 28: 13365, 29: 18478, 30: 19493, 31: 7576, 32: 10769, 33: 381, 34: 15111, 35: 17193, 36: 3466, 37: 8747, 38: 11960, 39: 17102, 40: 11849, 41: 14022, 42: 4528, 43: 1196, 44: 11671, 45: 1305, 46: 17997, 47: 10737, 48: 3377, 49: 6575, 50: 8822, 51: 11101, 52: 18033, 53: 375, 54: 5569, 55: 15581, 56: 9332, 57: 1, 58: 12517, 59: 852, 60: 10892, 61: 12431, 62: 18246, 63: 6632, 64: 11137, 65: 1007, 66: 17846, 67: 17323, 68: 5166, 69: 12975, 70: 6, 71: 6416, 72: 1254, 73: 6463, 74: 7451, 75: 15717, 76: 17008, 77: 2360, 78: 4067, 79: 3243, 80: 11307, 81: 5838, 82: 542, 83: 19105, 84: 15447, 85: 14049, 86: 12524, 87: 6648, 88: 7445, 89: 19140, 90: 13199, 91: 1473, 92: 9540, 93: 16260, 94: 6966, 95: 4515, 96: 4358, 97: 

In [479]:
test_data.replace('company', company_mapping)

,Unnamed: 0,latitude,longitude,company,is_local,type,fin_1,fin_2,fin_3,fin_4,target
0,160000,39.284310,-76.735350,18187,1,1,113033.389907,0.000000,270.906219,31222.780176,0
1,160001,36.758509,-76.344861,11208,0,0,-87239.590275,73759.387510,759.194862,237587.544996,1
2,160002,43.402802,-75.217100,7437,1,4,-122084.498620,15528.109943,1039.655934,29612.346982,0
3,160003,41.871160,-87.848570,17362,0,1,87355.127256,0.000000,214.594205,163526.475818,1
4,160004,42.161296,-88.129184,11515,0,3,33014.437946,0.000000,379.819724,67499.397999,1
...,...,...,...,...,...,...,...,...,...,...,...
39995,199995,35.241600,-80.983740,11640,0,3,194648.093228,0.000000,95.701485,37584.380280,1
39996,199996,35.241730,-80.983750,11849,0,3,-53202.007008,9641.082811,653.516413,189604.463353,0
39997,199997,35.290596,-80.756953,3987,0,3,-102398.568309,47105.929570,863.135873,26231.880089,0
39998,199998,35.204460,-80.720190,12287,0,0,32066.188506,6659.705140,363.861756,116288.412218,1


In [474]:
test_data.head

<bound method NDFrame.head of        Unnamed: 0   latitude  longitude  company  is_local  type  \
0          160000  39.284310 -76.735350    18187         1     1   
1          160001  36.758509 -76.344861    11208         0     0   
2          160002  43.402802 -75.217100     7437         1     4   
3          160003  41.871160 -87.848570    17362         0     1   
4          160004  42.161296 -88.129184    11515         0     3   
...           ...        ...        ...      ...       ...   ...   
39995      199995  35.241600 -80.983740    11640         0     3   
39996      199996  35.241730 -80.983750    11849         0     3   
39997      199997  35.290596 -80.756953     3987         0     3   
39998      199998  35.204460 -80.720190    12287         0     0   
39999      199999  35.222470 -80.847460        7         0     3   

               fin_1         fin_2        fin_3          fin_4  target  \
0      113033.389907      0.000000   270.906219   31222.780176       0   
1    

In [481]:
companies = train_data.groupby('company').count().iloc[:,0]
companies

company
-1        4565
 11         33
 20          3
 30         10
 46         18
          ... 
 19971      10
 19974      45
 19979      22
 19985      11
 19997       6
Name: Unnamed: 0, Length: 3683, dtype: int64

In [482]:
needed_companies = companies[companies > 10]
needed_companies

company
-1        4565
 11         33
 46         18
 61         34
 68        194
          ... 
 19958      13
 19960      14
 19974      45
 19979      22
 19985      11
Name: Unnamed: 0, Length: 1381, dtype: int64

In [483]:
def input_cluster(data):
    features = data[['latitude', 'longitude']]
    kmeans_clustering = KMeans(
    init="random",
    n_clusters=300,
    n_init=10,
    max_iter=300,
    random_state=42
    )
    kmeans_clustering.fit(features)
    labels = kmeans_clustering.labels_
    data['cluster'] = labels
    return data

In [484]:
def input_comp(data):
    data.loc[data.company <= 10, ('company')] = -1
    return data

In [485]:
def all_inputs(data):
    data = input_cluster(data)
    data = input_comp(data)

In [486]:
from sklearn.model_selection import train_test_split

In [487]:
train_data = train_data.drop(columns=train_data.columns[0])

In [488]:
train_data = train_data.drop(columns=['latitude', 'longitude'])

In [489]:
X, y = train_data.drop(columns=['target']), train_data['target']

In [490]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [492]:
X_train.shape

(144000, 8)

In [512]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [531]:
features = X_train[['fin_1', 'fin_2', 'fin_3', 'fin_4', 'is_local']]
company = X_train['company']
types = X_train['type']
location = X_train['cluster']
targets = y_train[y_train.columns[0]]
model.fit(
    {"features": features, "company": company, "type": types, "location": location},
    {"targets": targets},
    epochs=10,
    batch_size=128,
)

Train on 144000 samples
Epoch 1/10
   128/144000 [..............................] - ETA: 1:23:01

InvalidArgumentError:  indices[1,0] = 15581 is not in [0, 3683)
	 [[node model_23/embedding_45/embedding_lookup (defined at <ipython-input-531-c000603f0673>:10) ]] [Op:__inference_distributed_function_20910]

Errors may have originated from an input operation.
Input Source operations connected to node model_23/embedding_45/embedding_lookup:
 model_23/embedding_45/embedding_lookup/20155 (defined at D:\Anaconda3\lib\contextlib.py:112)

Function call stack:
distributed_function


In [529]:
print(y_train[y_train.columns[0]])

10548     1
40731     0
152747    1
140244    1
21626     1
         ..
97890     0
155390    1
138320    0
47694     0
36783     0
Name: target, Length: 144000, dtype: int64
